# Whisper ASR Pipeline

# Load Data

Agree to terms and conditions [here](https://huggingface.co/datasets/NathanRoll/Cambridge-Assessment) to download this gated dataset

In [ ]:
# https://huggingface.co/datasets/NathanRoll/Cambridge-Assessment/resolve/main/Cambridge%20Assessment%20Dataset-20240130T180103Z-001.zip
# https://huggingface.co/datasets/NathanRoll/Cambridge-Assessment/resolve/main/Cambridge%20Assessment%20Dataset-20240130T180103Z-002.zip

Unzip files, merge, and specify file path

In [ ]:
fpath = '/PATH_TO_UNZIPPED_FILES/'

# Load Modules

In [ ]:
!pip install praatio librosa
# !pip install g2p_en g2pM
!pip install jiwer

In [ ]:
import os
from os.path import exists, join, expanduser

import sys
import torch
from itertools import groupby
from jiwer import wer, mer

import pickle
import os
import pandas as pd
from tqdm import tqdm
from IPython.display import Audio
import numpy as np

In [ ]:
#create display audio function
def display_audio(filepath):
    return Audio(filepath)

# Data Formatting

In [ ]:
files = os.listdir(fpath)

There are 313 files in this dataset

In [ ]:
len({x.split('-')[0] for x in files if 'wav' in x})

313

Load first metadata source

In [ ]:
# Load the Excel file located at the provided file path (fpath) with the filename 'Annotated Files.xlsx' into an ExcelFile object
xls = pd.ExcelFile(fpath + 'Annotated Files.xlsx')

# Extract the names of all the sheets in the Excel file into the list 'sheet_list'
sheet_list = xls.sheet_names

# For each sheet in the Excel file, read its contents into a DataFrame and then concatenate all these DataFrames into one unified 'metadata' DataFrame
metadata = pd.concat([pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_list])

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


View metadata

In [ ]:
metadata

,File Name,Original Label,L1,Proficiency Level,Gender,Notes
0,Ar1/2-A2-M,S23R8UG2G8_SA_03 / S23R8UG2G8_SB_01,Arabic,A2,M,NaN
1,Po1/2-A2-F,S26PQUTURN_SA_03 / S26PQUTURN_SB_03,Polish,A2,F,NaN
2,Th1/2-C1-F,S35YQ66UN2_SA_04 / S35YQ66UN2_SB_01,Thai,C1,F,NaN
3,Fr1/2-B2-F,S368CHWSXV_SA_03 / S368CHWSXV_SB_01,French,B2,F,NaN
4,Ar3/4-C1-F,S38NMSPN5S_SA_03 / S38NMSPN5S_SB_01,Arabic,C1,F,NaN
...,...,...,...,...,...,...
14,Th47/48-A2-F,S6TFPZSQSN_SA_06 / S6TFPZSQSN_SB_04,Thai,A2,F,NaN
15,Viet51/52-A2-F,S6ZDZPPY6H_SA_06 / S6ZDZPPY6H_SB_05,Vietnamese,A2,F,NaN
16,Po55/56-C1-F,S72TU4TZRB_SA_05 / S72TU4TZRB_SB_03,Polish,C1,F,NaN
17,Th49/50-C1-F,S35YQ66UN2_SA_05 / S35YQ66UN2_SB_02,Thai,C1,F,NaN


Match metadata to files in dataset

In [ ]:
# Initialize an empty dictionary
d = {}

# Iterate over each row in the 'metadata' dataframe
for i, row in metadata.iterrows():
    # Get the 'File Name' value from the current row
    n = row['File Name']

    # Create a list named 'prefixes' containing the portion of the file name before the first '/'
    prefixes = [n.split('/')[0]]

    # Get the last character of the first item in the 'prefixes' list, and convert it to an integer
    c = int(prefixes[0][-1])

    # If the last character is '9'
    if c == 9:
        # If the second last character of the first item in the 'prefixes' list is alphabetical
        if prefixes[0][-2].isalpha():
            # Add '910' (i.e., '9' followed by '10') to the 'prefixes' list
            prefixes.append(prefixes[0][-1]+'10')
        else:
            # Get the last two characters, convert to integer, add 1, and then convert back to string.
            # Add this new value to the 'prefixes' list by replacing the last two characters of the first item.
            prefixes.append(prefixes[0][:-2] + str(int(prefixes[0][-2:])+1))
    else:
        # Replace the last character of the first item in the 'prefixes' list with its integer value plus 1,
        # then add this new value to the 'prefixes' list.
        prefixes.append(prefixes[0][:-1]+str(c+1))

    # Create a list of file names from the 'files' list that start with any of the prefixes followed by a '-'
    matching_files = [file for file in files if any(file.startswith(prefix+'-') for prefix in prefixes)]

    # Assign the 'matching_files' list to the 'n' key in the dictionary 'd'
    d[n] = matching_files

Create a seperate dictionary entry for each speaker, containing their respective files.

In [ ]:
# Iterate over each key-value pair in the dictionary 'd'
for k, v in d.items():

    # Extract the part of each filename before the first '.' (i.e., the stem of the filename) from the values in 'v' and store them in the list 'stems'
    stems = [x.split('.')[0] for x in v]

    # Initialize an empty dictionary 't'
    t = {}

    # For each stem in the 'stems' list
    for stem in stems:
        # Create a list of files from 'v' that start with the current 'stem'
        # and sort them. Assign this list to the key 'stem' in the 't' dictionary.
        t[stem] = sorted([x for x in v if x.startswith(stem)])

    # Update the value of key 'k' in dictionary 'd' with the dictionary 't'
    d[k] = t


# TextGrid

Create a function that extracts the word-level transcrip from each textgrid file

In [ ]:
def extract_transcript(path):
    """
    Extract the transcript from a given file based on specific delimiters.

    The function assumes that the transcript is located between `name = "Word"`
    and `name = "Syllable"`. The actual words are assumed to be in a format
    like `text = "word_here"`. The function then joins these words into a
    single string (transcript) and returns it.

    Parameters:
    - path (str): Path to the file from which to extract the transcript.

    Returns:
    - str: The extracted transcript.
    """

    # Open and read the file content
    with open(path, 'r') as f:
        content = f.read()

    # Extract the part of the content that lies between the defined delimiters and split it to get individual words
    words = content.split('''name = "Word"''')[-1].split('''name = "Syllable"''')[0].split('''text = "''')[1:]

    # Further split and extract actual words from the previous list
    words = [x.split('''"''')[0] for x in words]

    # Join the words into a transcript and remove double spaces
    transcript = ' '.join(words).replace('  ', ' ')

    # Remove leading and trailing spaces from the transcript
    if transcript[0] == ' ':
        transcript = transcript[1:]
    if transcript[-1] == ' ':
        transcript = transcript[:-1]

    return transcript

# Whisper

Install Whisper and acceleration libraries

In [ ]:
!pip install -q --upgrade torch torchvision torchaudio
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate optimum
!pip install -q ipython-autotime

import torch
from transformers import pipeline

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Load the three versions of the whisper large model @ fp16 precision

In [ ]:
# Optional: iterate over multiple versions of whisper
# models = ["openai/whisper-large","openai/whisper-large-v2","openai/whisper-large-v3"]

models = ["openai/whisper-large-v3"]

d_models = {}

for m in models:
  d_models[m] = pipeline("automatic-speech-recognition",
                m,
                torch_dtype=torch.float16,
                device="cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Define the transcription function that returns Whisper's text inference from the audio input.

In [ ]:
def transcribe(audio):
    outputs = {}

    r = d_models[models[-1]](audio,
                    chunk_length_s=30,
                    batch_size=24,
                    return_timestamps=True)['text']
    return r

Ensure that textgrid transcript matches audio

In [ ]:
_ = sorted(d['Fr43/44-A2-F']['Fr43-A2-F'])

print('gold transcript: ' + extract_transcript(fpath+_[0]))

transcribe(fpath+'Fr43-A2-F.wav')

In [ ]:
display_audio(fpath+_[1])

Generate nested dictionary comparing whisper transcription and manual transcription.

In [ ]:
# Iterate over each key-value pair in the dictionary 'd',
# displaying a progress bar using 'tqdm' for better user feedback during the loop's execution
for k, v in tqdm(d.items()):

    # For each inner key-value pair (stem and fileset) in the value 'v'
    for stem, fileset in v.items():

        # If the fileset has exactly 2 items, it's assumed that one is a Whisper transcript and the other is a manual transcript
        if len(fileset) == 2:

            # Transcribe the second file in the fileset using the 'transcribe' function
            whisper_transcript = transcribe(fpath + fileset[1])

            # Extract the transcript from the first file in the fileset using the 'extract_transcript' function
            manual_transcript = extract_transcript(fpath + fileset[0])

            # Update the dictionary 'd' at keys 'k' and 'stem' to hold both the manual and Whisper transcripts as a list
            d[k][stem] = [manual_transcript, whisper_transcript]

        # If the fileset doesn't have exactly 2 items, print the fileset and set both transcripts to '(MISSING_DATA)'
        else:
            print(fileset)
            d[k][stem] = ['(MISSING_DATA)', '(MISSING_DATA)']


# WER

Generate WER and MER for each Whisper/manual transcript pair

In [ ]:
# Initialize two dictionaries to store Word Error Rate (WER) and Matched Error Rate (MER) results for each speaker
wer_results = {}
mer_results = {}

# Iterate over each speaker and their associated transcripts in the dictionary 'd'
for speaker, transcripts in d.items():

    # Initialize empty dictionaries for the current speaker in both the WER and MER results dictionaries
    wer_results[speaker] = {}
    mer_results[speaker] = {}

    # For each sample and its associated sentences (transcripts) in the current speaker's transcripts
    for sample, sentences in transcripts.items():

        # Extract the ground truth transcript from the sentences. Remove any '(?)' and '(MISSING_DATA)' placeholders.
        ground_truth = sentences[0].replace('(?)','').replace('(MISSING_DATA)','')

        # Extract the hypothesis (possibly the machine-generated transcript)
        hypothesis = sentences[1]

        try:
            # Calculate the WER and MER between the ground truth and the hypothesis
            error_wer = wer(ground_truth, hypothesis)
            error_mer = mer(ground_truth, hypothesis)
        except:
            # If there's an error in calculating WER or MER, set the results to NaN (Not a Number)
            error_wer = np.nan
            error_mer = np.nan

        # Store the WER and MER results for the current sample under the current speaker's entry
        wer_results[speaker][sample] = error_wer
        mer_results[speaker][sample] = error_mer

Structure results at the speaker level

In [ ]:
# Iterate over the WER results for each speaker
for k, v in wer_results.items():
    # For each speaker, compute the mean of their WER results across all samples
    # Using 'np.nanmean' ensures that any NaN values are ignored in the computation
    wer_results[k] = np.nanmean(list(v.values()))

# Iterate over the MER results for each speaker
for k, v in mer_results.items():
    # For each speaker, compute the mean of their MER results across all samples
    # Using 'np.nanmean' ensures that any NaN values are ignored in the computation
    mer_results[k] = np.nanmean(list(v.values()))

<ipython-input-39-f220d17292e3>:5: RuntimeWarning: Mean of empty slice
  wer_results[k] = np.nanmean(list(v.values()))
<ipython-input-39-f220d17292e3>:11: RuntimeWarning: Mean of empty slice
  mer_results[k] = np.nanmean(list(v.values()))


Generate dataframes for both outputs and merge them with the metadata

In [ ]:
# Convert the 'wer_results' dictionary into a DataFrame with the index named 'WER'
# Transpose the DataFrame (using '.T') so that each speaker becomes a row and 'WER' becomes a column
res = pd.DataFrame(wer_results, index=['WER']).T

# Reset the index of the 'res' DataFrame, moving the index (the speakers) into a column
res.reset_index(inplace=True)

# Rename the columns of the 'res' DataFrame for clarity
res.columns = ['File Name', 'WER']

# Convert the 'mer_results' dictionary into a DataFrame with the index named 'MER'
# Transpose the DataFrame (using '.T') so that each speaker becomes a row and 'MER' becomes a column
res_mer = pd.DataFrame(mer_results, index=['MER']).T

# Reset the index of the 'res_mer' DataFrame, moving the index (the speakers) into a column
res_mer.reset_index(inplace=True)

# Rename the columns of the 'res_mer' DataFrame for clarity
res_mer.columns = ['File Name', 'MER']

# Merge the 'metadata' DataFrame with the 'res' DataFrame on the 'File Name' column
# If a 'File Name' in 'metadata' doesn't exist in 'res', it will still be kept in the result, with NaN for the 'WER' value (due to 'how='outer'')
result = pd.merge(metadata, res, how='outer', on='File Name')

# Merge the newly created 'result' DataFrame with the 'res_mer' DataFrame on the 'File Name' column
# If a 'File Name' in 'result' doesn't exist in 'res_mer', it will still be kept in the result, with NaN for the 'MER' value (due to 'how='outer'')
result = pd.merge(result, res_mer, how='outer', on='File Name')

Export results

In [ ]:
result.to_csv('CAM_ASSESSMENT_WER_MER.csv')

WER matches MER ~75% of the time

In [ ]:
np.nanmean(result['WER']==result['MER'])

0.7452229299363057

# Match Cam Assessment data to Proficiency Score

In [ ]:
# Load a CSV file into a DataFrame
df_processed = pd.read_csv('/content/CAM_ASSESSMENT_WER_MER_v3.csv')

# Create a new column 'token_match' in the DataFrame by extracting the first token before the underscore from the 'Original Label' column
df_processed['token_match'] = df_processed['Original Label'].apply(lambda x: x.split('_')[0])

# Drop the column named 'Unnamed: 0' from the DataFrame
df_processed.drop('Unnamed: 0', inplace=True, axis=1)

# Open and read the contents of a text file into a list of lines
with open('/content/drive/Shareddrives/CAM_ASSESSMENT_CONF/pdf_extract.txt','r') as f:
    text = f.readlines()

# Remove the substring ' High' from each line in the text
text = [x.replace(' High','') for x in text]

# Process the list of text lines to extract relevant components
processed_data = []
for d in text:
    components = d.strip().split()
    # If the line has 13 components, append "None" to handle any missing "Extra Note"
    if len(components) == 13:
        components.append(None)
    processed_data.append(components)

# Convert the processed data into a DataFrame with specified column names
df_meta = pd.DataFrame(processed_data, columns=["token_match", "Country", "First Langu", "Gender", "YOB", "Test Version", "Speaking", "Part 1", "Part 2", "Part 3", "Part 4", "Part 5", "Total", "Transcribed_by"])

# Check the unique values in the 'token_match' column of the df_meta DataFrame
df_meta['token_match'].unique

# Merge the df_processed and df_meta DataFrames based on the 'token_match' column using an outer join
merged_df = df_processed.merge(df_meta, on='token_match', how='outer')

# Calculate the mean of NaN values in the 'File Name' column of the merged DataFrame
np.mean(merged_df['File Name'].isna())

# Save the merged DataFrame to a CSV file
merged_df.to_csv('cambrdige_WERMER_merged_meta_fixed.csv')

# Group the merged DataFrame by 'token_match' and compute the mean values for 'WER' and 'MER' columns
merged_df.groupby('token_match').mean()[['WER','MER']]

# Group the merged DataFrame by 'token_match', extract the first row for each group, drop 'WER' and 'MER' columns, and join it with the mean 'WER' and 'MER' values calculated in the previous step. Then, save the resulting DataFrame to a CSV file.
merged_df.groupby('token_match').first().drop(['WER','MER'],axis=1)\
  .join(merged_df.groupby('token_match').mean()[['WER','MER']])\
  .to_csv('speaker_level_WER_MER.csv')


In [ ]:
df_working = merged_df.groupby('token_match').first().drop(['WER','MER'],axis=1)\
  .join(merged_df.groupby('token_match').mean()[['WER','MER']])

Save merged dataframe as a CSV

In [ ]:
df_working[df_working['MER'].notna()].to_csv('Cambridge_v3_Fixed.csv')

# IViE

Define a function to transcribe audio from an audio array

In [ ]:
def transcribe_from_array(audio_array):
    # If the length of the audio in seconds is greater than 30, print a warning
    if len(audio_array)/16000 > 30:
        print('warning: audio too long')

    # Pad or trim the audio array to a desired length
    audio = whisper.pad_or_trim(audio_array)

    # Convert the audio into a mel spectrogram and transfer it to the model's device (e.g., GPU)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # Set decoding options for the whisper transcription
    options = whisper.DecodingOptions(language='en')

    # Use the whisper model to transcribe the mel spectrogram
    result = whisper.decode(model, mel, options)

    # Return the transcribed text
    return result.text

Define a function to calculate WER (Word Error Rate) and MER (Matched Error Rate) for a given row

In [ ]:
def calculate_wer_and_mer(row):
    ground_truth = row['actual_transcript']
    hypothesis = row['whisper_generated']

    row['WER'] = wer(ground_truth, hypothesis)
    row['MER'] = mer(ground_truth, hypothesis)

    return row

Load the IViE dataset from a Pickle file located on a Google Drive shared drive

In [ ]:
with open('/content/drive/Shareddrives/Ling Thesis/SBC_TEST_IViE.pkl', 'rb') as f:
    ivie = pickle.load(f)

Play the audio from the first row of the 'ivie' DataFrame

In [ ]:
Audio(ivie['audio'].iloc[0], rate=16000)

Transcribe the audio from the first row of the 'ivie' DataFrame

In [ ]:
transcribe(ivie['audio'].iloc[0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


' But then all of a sudden the fairy godmother came from nowhere and said Cinderella I will make you go to the ball as beautiful as you can be and she sent her out into the garden'

Create 'actual transcript'

In [ ]:
# Transcribe all audio samples in the 'ivie' DataFrame and store the results in a new column 'whisper_generated'
ivie['whisper_generated'] = [transcribe(ivie['audio'].iloc[i].astype(np.float32)) for i in range(len(ivie))]

# Keep only the 'actual_transcript' and 'whisper_generated' columns in the 'ivie' DataFrame
ivie = ivie[['actual_transcript','whisper_generated']]

# Apply a replace operation on the 'actual_transcript' column
ivie['actual_transcript'] = ivie['actual_transcript'].apply(lambda x: x.replace('<|IU_Boundary|>',''))

<ipython-input-72-d796425a58ec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ivie['actual_transcript'] = ivie['actual_transcript'].apply(lambda x: x.replace('<|IU_Boundary|>',''))


Merge with Metadata

In [ ]:
# Apply the 'calculate_wer_and_mer' function to each row of the 'ivie' DataFrame
ivie = ivie.apply(calculate_wer_and_mer, axis=1)

# Extract metadata (e.g., variety, sex) from the index of the 'ivie' DataFrame
metadata = [x.split('_') for x in ivie.index]

# Create new columns for 'variety' and 'sex' based on the extracted metadata
ivie['variety'] = [x[0] for x in metadata]
ivie['sex'] = [x[1][0] for x in metadata]

Recompute IViE scores at the speaker level > save to CSV

In [ ]:
# Group the 'ivie' DataFrame by 'variety' and 'sex' and compute the mean of the WER and MER values for each group
ivie_wer_mer = ivie.groupby(['variety','sex']).mean()

# Reset the index of the 'ivie_wer_mer' DataFrame
ivie_wer_mer = ivie_wer_mer.reset_index()

# Save the 'ivie_wer_mer' DataFrame to a CSV file
ivie_wer_mer.to_csv('IVIE_SPONTANEOUS_WER_MER.csv')

<ipython-input-74-e68594e19511>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ivie_wer_mer = ivie.groupby(['variety','sex']).mean()


# Speech Accent Archive [link](https://accent.gmu.edu/browse_language.php)

Extract file paths from SAA

In [ ]:
saa_files = pd.read_html('https://accent.gmu.edu/soundtracks/')[0]
saa_files = list(saa_files['Name'])
saa_files = [x for x in saa_files if '.mp3' in str(x)]

Define stem of audio files

In [ ]:
path_stem = 'https://accent.gmu.edu/soundtracks/'

Transcribe each file in the SAA

In [ ]:
out = {}

for f in tqdm(saa_files):
  out[f] = transcribe(path_stem + f)

Streaming output truncated to the last 5000 lines.
  warnings.warn(
 18%|█▊        | 538/3011 [26:45<1:50:48,  2.69s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 18%|█▊        | 539/3011 [26:47<1:51:13,  2.70s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 18%|█▊        | 540/3011 [26:50<1:51:39,  2.71s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 18%|█▊        | 541/3011 [26:53<1:52:21,  2.73s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101:

Save transcripts

In [ ]:
with open('/content/drive/Shareddrives/Ling Thesis/Data_Dir/SAA_Whisperv3.pkl', 'wb') as f:
  pickle.dump(out, f)

## Load for SAA

Load transcripts

In [ ]:
with open('/content/drive/Shareddrives/Ling Thesis/Data_Dir/SAA_Whisperv3.pkl', 'rb') as f:
  saa_transcripts = pickle.load(f)

View example

In [ ]:
saa_transcripts['english489.mp3']

' Please call Stella. Ask her to bring these things with her from the store. Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob. We also need a small plastic snake and a big toy frog for the kids. She can school these things into three red bags and we will go meet her Wednesday at the train station.'

Define elicitation passage for WER/MER calculations

In [ ]:
elicitation = 'Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.'

Create a dataframe and add transcripts, WER, and MER

In [ ]:
df = pd.DataFrame()
df.index = saa_transcripts.keys()
df['whisper'] = saa_transcripts.values()
df['wer'] = [wer(elicitation, x) for x in saa_transcripts.values()]
df['mer'] = [mer(elicitation, x) for x in saa_transcripts.values()]
df

,whisper,wer,mer
afrikaans1.mp3,"Please call Stella, ask her to bring these th...",0.072464,0.072464
afrikaans2.mp3,Please call Stella. Ask her to bring these th...,0.043478,0.043478
afrikaans3.mp3,Please call Stella. Ask her to bring these th...,0.043478,0.043478
afrikaans4.mp3,Please call Stella. Ask her to bring these th...,0.028986,0.028986
afrikaans5.mp3,Please call Stella. Ask her to bring these th...,0.057971,0.057143
...,...,...,...
yoruba8.mp3,Please call Stella. Ask her to bring these th...,0.115942,0.115942
yupik1.mp3,Please call Stella. Ask her to bring these th...,0.028986,0.028986
yupik2.mp3,Please call Stella. Ask her to bring these th...,0.217391,0.182927
yupik3.mp3,Please call Stella. Ask her to bring these th...,0.028986,0.028986


#SAA Metadata

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import trange

In [ ]:
outs = {}

for i in trange(1,3036):
  url = f'https://accent.gmu.edu/browse_language.php?function=detail&speakerid={i}'
  r = requests.get(url)

  html = BeautifulSoup(r.content)
  t = [x.text for x in html.find_all('li') if '</em>' in str(x)]
  t = {x.split(':')[0]:x.split(':')[1][1:] for x in t}
  t['features'] = [x.text for x in html.findAll('a') if 'browse_language.php?function=detail&amp' in str(x)][1:]

  t['index'] = i
  n = html.findAll('h5')[1].em.text
  outs[n] = t

100%|██████████| 3035/3035 [46:19<00:00,  1.09it/s]


In [ ]:
with open('/content/drive/Shareddrives/Ling Thesis/Data_Dir/SAA_metadata.pkl', 'wb') as f:
  pickle.dump(outs, f)

In [ ]:
df['to_join'] = [x[0] for x in df.index.str.split('.')]

In [ ]:
df_meta = pd.DataFrame(outs).T

In [ ]:
df_meta['to_join'] = df_meta.index

In [ ]:
df_merged = df.merge(df_meta, on='to_join', how='left')

In [ ]:
df_merged.index = df_merged['to_join']
df_merged.fillna(' , ', inplace=True)
df_merged['birth place'] = df_merged['birth place'].apply(lambda x: str(x).replace(' (map)',''))
df_merged['native language'] = df_merged['native language'].apply(lambda x: str(x).split('\n')[0])
df_merged['length of english residence'] = df_merged['length of english residence'].apply(lambda x: str(x).split(' ')[0])

df_merged['age'] = df_merged['age, sex'].apply(lambda x: str(x).split(', ')[0])
df_merged['sex'] = df_merged['age, sex'].apply(lambda x: str(x).split(', ')[1])

In [ ]:
df_merged = df_merged.drop(['whisper','age, sex', 'to_join'], axis=1)
df_merged.to_csv('SAA_full.csv')

In [ ]:
df_merged

,wer,mer,birth place,native language,other language(s),age of english onset,english learning method,english residence,length of english residence,features,index,age,sex
to_join,,,,,,,,,,,,,
afrikaans1,0.072464,0.072464,"virginia, south africa",afrikaans,tswana,9,academic,usa,0.5,"[final obstruent devoicing, interdental fricat...",1,27,female
afrikaans2,0.043478,0.043478,"pretoria, south africa",afrikaans,dutch german french,5,academic,usa,10,"[final obstruent devoicing, vowel shortening]",2,40,male
afrikaans3,0.043478,0.043478,"pretoria, transvaal, south africa",afrikaans,gujarati sindhi french,4,naturalistic,usa,20.7,"[final obstruent devoicing, r to trill, non as...",418,43,male
afrikaans4,0.028986,0.028986,"pretoria, south africa",afrikaans,flemish dutch,8,academic,usa,15,[],1159,26,male
afrikaans5,0.057971,0.057143,"cape town, south africa",afrikaans,none,6,academic,australia,9,[],1432,19,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...
yoruba8,0.115942,0.115942,"ibadan, oyo, nigeria",yoruba,none,2,academic,"nigeria, usa",21,[],2985,21,male
yupik1,0.028986,0.028986,"bethel, alaska, usa",yupik,spanish,1,naturalistic,alaska,31,"[final obstruent devoicing, interdental fricat...",571,31,female
yupik2,0.217391,0.182927,"nome, alaska, usa",yupik,russian,6,academic,usa,40,[],2637,40,male
